Test notebook


In [ ]:
# %%

import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import numpy as np


In [ ]:

''' Format plots'''

fsize = 11
tsize = 18
tdir = 'in'
major = 5.0
minor = 3.0
lwidth = 0.8
lhandle = 2.0
plt.style.use('default')
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = fsize
plt.rcParams['legend.fontsize'] = fsize
plt.rcParams['xtick.direction'] = tdir
plt.rcParams['ytick.direction'] = tdir
plt.rcParams['xtick.major.size'] = major
plt.rcParams['xtick.minor.size'] = minor
plt.rcParams['ytick.major.size'] = 5.0
plt.rcParams['ytick.minor.size'] = 3.0
plt.rcParams['axes.linewidth'] = lwidth
plt.rcParams['legend.handlelength'] = lhandle
plt.rcParams['axes.autolimit_mode'] = 'round_numbers'
plt.rcParams['axes.xmargin'] = 0
plt.rcParams['axes.ymargin'] = 0
plt.rcParams['figure.dpi'] = 300
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=["yellowgreen", "black", "dimgrey", 'darkgreen', 'aqua', 'deepskyblue', 'orange', 'blue', 'indianred']) 
plt.rcParams['figure.figsize'] = [14/2.54, 10/2.54]



Import the data from the runs 

In [ ]:
# all_strike_prices = np.load(f'C:/Users/LukaK/OneDrive - Durham University/Projects/Agent based modelling easy/PowerSim/data_out/all_strike_prices.npy')
# total_per_tech = np.load(f'C:/Users/LukaK/OneDrive - Durham University/Projects/Agent based modelling easy/PowerSim/data_out/total_per_tech.npy')

import os

name = 'more_storage_no_variance'
directory = f'data/{name}/'
''' set the number parameters from the run'''
n_days = 8
n_years = 30


descriptor  = f'Graphs/{name}'
tech_types = ['nuclear', 'coal', 'fossil_fuel', 'bioenergy', 'wind_onshore', 'wind_offshore', 'solar', 'hydro', 'CCGT']

npy_files = [filename for filename in sorted(os.listdir(directory)) if filename.endswith('total_per_tech.npy')]

# Load the first file to initialize the array


arrays =[]
# stack the remaining files
for filename in npy_files:
    filepath = os.path.join(directory, filename)
    file_data = np.load(filepath)
    arrays.append(file_data)

total_per_tech = np.stack(arrays, axis =0)
'''[run, year, tech, hour]'''


print(np.shape(total_per_tech))

'''get storage data'''
storage_file_names = [filename for filename in sorted(os.listdir(directory)) if filename.endswith('storage_plant_prod.csv')]
arrays = []
for filename in storage_file_names:
    df = pd.read_csv(os.path.join(directory, filename))

    battery_cols = df.filter(regex='battery', axis=1)
    battery_sum = battery_cols.sum(axis=1).values
    battery_sum_pos = battery_sum.copy().clip(min=0)
    battery_sum_neg = battery_sum.copy().clip(max=0)
    battery_sum_pos = np.array(battery_sum_pos).reshape(int(n_years), -1)
    battery_sum_neg = np.array(battery_sum_neg).reshape(int(n_years), -1)


    pumped_hydro_cols = df.filter(regex='pumped_hydro', axis=1)
    pumped_hydro_sum = pumped_hydro_cols.sum(axis=1).values
    pumped_hydro_sum_pos = pumped_hydro_sum.copy().clip(min=0)
    pumped_hydro_sum_neg = pumped_hydro_sum.copy().clip(max=0)
    pumped_hydro_sum_pos = np.array(pumped_hydro_sum_pos).reshape(int(n_years), -1)
    pumped_hydro_sum_neg = np.array(pumped_hydro_sum_neg).reshape(int(n_years), -1)

    total = np.stack([battery_sum_neg, pumped_hydro_sum_neg, battery_sum_pos,pumped_hydro_sum_pos])
    arrays.append(total)

total_per_storage = np.stack(arrays)
total_per_storage = total_per_storage.transpose(0,2, 1,3)
print(f'storage shape {total_per_storage.shape}')
total = np.concatenate((total_per_storage, total_per_tech), axis = 2)
print(f'total shape {total.shape}')

'''Change the data'''
from scipy.stats import norm

x = range(1, n_years+1)
data = np.mean(total_per_tech, axis = 3)  # 10 sets of 100 data points

print(data.shape)

# Generate some sample data


# Loop through each of the 5 sets of data
mean_all = []
stdev_all = []
for j in range(data.shape[2]):
    means = []
    stdev = []
    # Loop through each of the 10 sets of data within the current set
    for i in range(data.shape[1]):
        y = data[:, i, j]
        mu, std = norm.fit(y)
        means.append(mu)
        stdev.append(std)
    # Calculate the average parameters and errors for the current set
   
    # Add the average parameters and errors to the list for all sets
    mean_all.append(means)
    stdev_all.append(stdev)

mean_all = np.array(mean_all)
stdev_all = np.array(stdev_all)
# Calculate the average parameters and errors for all sets

# Print the results
print(mean_all.shape)

Make the graphs

In [ ]:


'''Stacked plot of daily production'''

year = 0

ydata = total[1,year,2:,:]/1000

ydata2 = total[1,year,:2,:]/1000
print(np.shape(ydata))

xdata = np.arange(24, (n_days+1)*24)/24


mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=['red', 'green', "yellowgreen", "black", "dimgrey", 'darkgreen', 'aqua', 'deepskyblue', 'orange', 'blue', 'indianred']) 
fig, ax = plt.subplots()

# print(ydata)
tech_types_named =  ['Battery', 'Pumped hydro', 'Nuclear', 'Coal', 'Other fossil fuel','Biomass', 'Onshore wind', 'Offshore wind', 'Solar', 'Hydro', 'Gas']
ax.stackplot(xdata,*ydata, labels = [t for t in tech_types_named], baseline = 'zero')

'''plot the storage production'''
# ax.stackplot(xdata,*ydata2)
#have to remove the historical data from the plot



ax.set_ylabel('Electricity production (GW)')
ax.set_xlabel('Day')
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.xaxis.set_major_locator(MultipleLocator(1))
ax.set_xlim(1, max(xdata))
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.legend(loc = (1.04, 0.25))
# ax.set_xlim(0, n_years)
# ax.set_ylim(min(average_yearly_strike_prices) - 5, (max(average_yearly_strike_prices) + 5))
ax.spines.right.set_visible(False)
ax.spines.top.set_visible(False)

plt.savefig(fname = f'{descriptor}year production by day', bbox_inches = "tight" )
plt.show()



In [ ]:


fig, ax = plt.subplots()

total_per_tech_average= np.mean(total_per_tech, axis=3)

ydata = np.mean(total_per_tech_average, axis = 0)
print(np.shape(ydata))
xdata = range(1, n_years+1)

# print(ydata)
tech_types_named =  ['Nuclear', 'Coal', 'Other fossil fuel','Biomass', 'Onshore wind', 'Offshore wind', 'Solar', 'Hydro', 'Gas']
colours = ["yellowgreen", "black", "dimgrey", 'darkgreen', 'aqua', 'deepskyblue', 'orange', 'blue', 'indianred']
for i in range(9):
    y = ydata[:,i]/(960)
    ax.plot(xdata,y, label =  tech_types_named[i], color = colours[i])
    ax.plot(xdata, (mean_all[i,:] + 2*stdev_all[i, :])/960, color = colours[i], alpha = 0.1)
    ax.plot(xdata, (mean_all[i,:] - 2*stdev_all[i, :])/960, color = colours[i], alpha = 0.1)
    ax.fill_between(xdata, (mean_all[i,:] + 2*stdev_all[i, :])/960, (mean_all[i,:] - 2*stdev_all[i, :])/960,color = colours[i], alpha = 0.1)
#have to remove the historical data from the plot

ax.set_ylabel('Average electricity production (GW)')
ax.set_xlabel('Year')
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.xaxis.set_major_locator(MultipleLocator(5))
ax.set_xlim(1, max(xdata))
ax.set_ylim(0)
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.legend(loc = (1.04, 0.25))
# ax.set_xlim(0, n_years)
# ax.set_ylim(min(average_yearly_strike_prices) - 5, (max(average_yearly_strike_prices) + 5))
ax.spines.right.set_visible(False)
ax.spines.top.set_visible(False)

plt.savefig(fname = f'{descriptor}average_production yearly', bbox_inches = "tight" )
plt.show()




In [ ]:

'''Get average yearly prices'''


npy_files = [filename for filename in os.listdir(directory) if filename.endswith('all_strike_prices.npy')]

# Load the first file to initialize the array


arrays =[]
# stack the remaining files
for filename in npy_files[1:]:
    filepath = os.path.join(directory, filename)
    file_data = np.load(filepath)
    arrays.append(file_data)

yearly_price = np.stack(arrays, axis =0)
'''[run, day, hour]'''

yearly_price.shape

In [ ]:
n_days = 8
n_runs = np.shape(yearly_price)[0]
print(np.shape(yearly_price))
print(yearly_price)


average_daily_p = np.mean(yearly_price, axis = 2)

av_daily_2d = average_daily_p.reshape(n_runs,n_years,n_days)
print(np.shape(av_daily_2d))

mean_yearly = np.mean(av_daily_2d, axis = 2)


x = range(1, n_years+1)
data = mean_yearly  # 10 sets of 100 data points

print(np.shape(data))

# Generate some sample data


# Loop through each of the 5 sets of data
mean_all_price = []
stdev_all_price = []

for i in range(data.shape[1]):
    y = data[:, i]
    mu, std = norm.fit(y)
    mean_all_price.append(mu)
    stdev_all_price.append(std)
# Calculate the average parameters and errors for the current set

# Add the average parameters and errors to the list for all sets


mean_all_price = np.array(mean_all_price)
stdev_all_price = np.array(stdev_all_price)
# Calculate the average parameters and errors for all sets

# Print the results
print(mean_all_price)
print(stdev_all_price)
mean_all.shape


In [ ]:

fig, ax = plt.subplots()


ydata = mean_all_price

xdata = range(1, n_years+1)

# print(ydata)
tech_types_named =  ['Nuclear', 'Coal', 'Other fossil fuel','Biomass', 'Onshore wind', 'Offshore wind', 'Solar', 'Hydro', 'Gas']

ax.plot(xdata,ydata, color = 'r')
ax.plot(xdata, ydata + 2*stdev_all_price, color = 'r', alpha = 0.1)
ax.plot(xdata, ydata - 2*stdev_all_price, color = 'r', alpha = 0.1)
ax.fill_between(xdata, ydata + 2*stdev_all_price, ydata - 2*stdev_all_price, color ='r', alpha = 0.1 )
#have to remove the historical data from the plot

ax.set_ylabel('Average electricity price (GBP)')
ax.set_xlabel('Year')
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.xaxis.set_major_locator(MultipleLocator(5))
ax.set_xlim(1, max(xdata))
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.legend(loc = (1.04, 0.25))
# ax.set_xlim(0, n_years)
# ax.set_ylim(min(average_yearly_strike_prices) - 5, (max(average_yearly_strike_prices) + 5))
ax.spines.right.set_visible(False)
ax.spines.top.set_visible(False)

plt.savefig(fname = f'{descriptor}av. price.png', bbox_inches = "tight" )
plt.show()


In [ ]:
import electricity_company
import pickle

with open('powerplants.pickle', 'rb') as f:
    obj_list_new = pickle.load(f)


In [ ]:


tech_types_named =  ['Nuclear', 'Coal', 'Other fossil fuel','Biomass', 'Onshore wind', 'Offshore wind', 'Solar', 'Hydro', 'Gas']

for obj in obj_list_new: 
    if obj.name == 'RWE Npower':
        elec_co = obj

plant_list_npower = elec_co.power_plants

for plant in plant_list:
    print(plant.capacity_MW)

    for tech in tech_types:
        energy = [(plant.energy_supplied_per_hour[-(24*world.n_days):]) for plant in plant_list if plant.technology == tech and len(plant.energy_supplied_per_hour) >= 24]
        energy = np.sum(energy, axis = 0)
        y_total_per_tech.append(energy)

In [ ]:
import os

directory = 'data/total_per_tech/7almostworking/monies/'

npy_files = [filename for filename in os.listdir(directory) if filename.endswith('.npy')]

# Load the first file to initialize the array


arrays =[]
# stack the remaining files
for filename in npy_files[1:]:
    filepath = os.path.join(directory, filename)
    file_data = np.load(filepath)
    arrays.append(file_data)

monies = np.stack(arrays, axis =0)
'''[run, year, tech, hour]'''

print(monies[6,:])